<a href="https://colab.research.google.com/github/Avatar2001/VectorDatabase/blob/main/Vecto_databases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets faiss-cpu==1.7.4 chromadb==0.4.22 sentence-transformers==2.3.1

In [ ]:
!pip install -U datasets

In [ ]:
from datasets import load_dataset

dataset=load_dataset("sadeem-ai/arabic-qna")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset

In [ ]:
dataset=dataset.filter(lambda example: example['has_answer'] == True)

In [ ]:
dataset

In [ ]:
text=dataset['train']['text']
text[0]

In [ ]:
metadata=[
    {
        "source":record['source'],
        "title":record['title']
    }

          for record in dataset['train']

]
metadata[0]

In [ ]:
ids=[
    str(i)
    for i in range(len(text))
]



In [ ]:
from sentence_transformers import SentenceTransformer
model_id="sentence-transformers/distiluse-base-multilingual-cased-v2"
device="cuda:0"
dim=512
model=SentenceTransformer(model_id,device=device)

In [11]:
encode_doc=model.encode(text,show_progress_bar=True)

Batches:   0%|          | 0/127 [00:00<?, ?it/s]

In [12]:
encode_doc.shape

(4037, 512)

In [13]:
len(text)

4037

In [14]:
text[0],encode_doc[0]

('اعتمادًا على المكان الذي ينتمي إليه الفرد، قد يكون تصورهم للوقت هو أن «الساعة تحكم اليوم» أو «سنصل إلى هناك عندما نصل إلى هناك». تحسين احتمالات النجاح في المجتمع العالمي يتطلب فهم الاختلافات الثقافية والتقاليد وأنماط الاتصال.',
 array([-2.22630110e-02,  1.18622780e-02, -5.17282784e-02, -3.84508967e-02,
         1.07650623e-01, -4.56619374e-02,  5.49685694e-02,  9.17534716e-03,
         3.52448076e-02,  6.96124462e-03,  3.18153016e-02,  2.13808799e-03,
        -9.56622139e-03, -6.63584918e-02, -4.39523384e-02,  8.27048048e-02,
         1.58673935e-02, -7.34318560e-03, -2.98200361e-02, -3.95070873e-02,
        -1.50665967e-02,  2.46608946e-02,  7.54381856e-03, -2.70484686e-02,
        -4.28158455e-02,  1.73906249e-03, -3.48085142e-03,  8.00499506e-03,
         1.49304494e-02, -1.12245372e-03, -8.69526118e-02,  2.12956294e-02,
         1.40109390e-03, -3.78256552e-02, -1.58398207e-02,  3.27752344e-02,
         4.64597298e-03,  5.33918887e-02,  1.11280270e-02,  1.02619836e-02,
        -2

In [15]:
!pip install numpy==1.26.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 85.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.


In [ ]:
import chromadb

chroma_client = chromadb.PersistentClient(path=r"D:\chroma-ar-docs")

In [ ]:
collection=chroma_client.create_collection(
    name='Documnts',
    metadata={"hnsw:space":"cosine"}

)

In [ ]:
collection.add(
    documents=text,
    embeddings=encode_doc,
    metadatas=metadata,
    ids=ids

)

In [ ]:
question='ماذا كانت ترغب إسبانيا في استعادته في عام 1814؟'
question_embd=model.encode(question)
results=collection.query(
    query_embeddings=question_embd.tolist(),
    n_results=3

)
results

In [ ]:
import faiss
import numpy as np
from copy import deepcopy

In [ ]:
norm_encode_docs=deepcopy(encode_doc)
faiss.normalize_L2(norm_encode_docs)

In [ ]:
faiss_index=faiss.IndexIDMap(faiss.IndexFlatIP(dim))
faiss_index.add_with_ids(norm_encode_docs,ids)

In [ ]:
question1='ماذا كانت ترغب إسبانيا في استعادته في عام 1814؟'
question_embd1=model.encode([question1])
faiss.normalize_L2(question_embd1)
result=faiss_index.search(question_embd1,3)
result

In [ ]:
text[139]

In [ ]:
import pickle

with open("./Faiss_arabic_document/index.pickle","wb") as handle:
  pickle.dump(faiss_index,handle,protocol=pickle.HIGHEST_PROTOCOL)

with open("./Faiss_arabic_document/data.pickle","wb") as handle:
  pickle.dump({

          "text":text,
          "ids":ids,
          "metadata":metadata
      },handle,protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
with open("./Faiss_arabic_document/index.pickle","rb") as handle:
  loaded_faiss_data=pickle.load(handle)

with open("./Faiss_arabic_document/data.pickle","rb") as handle:
  loaded_data=pickle.load(handle)